In [4]:
def import_ontology_to_neo4j():
    import os
    import logging
    
    from neo4j import GraphDatabase
    
    url = "bolt://0.0.0.0:7687"
    username = "neo4j"
    password = "neo4j"
    driver = GraphDatabase.driver(url, auth=(username, password))

    def queryNeo4j(driver, query):
        """Runs a single Cypher query."""
        with driver.session() as session:
            try:
                session.run(query)
            except Exception as e:
                pass

    importQuery = 'CALL n10s.rdf.import.fetch("file:///home/lukas/CS425/Automatic_Taxonomy_Construction_NNs/data/userinput/user_owl.owl", "RDF/XML");'
    queryNeo4j(driver, "MATCH(n) DETACH DELETE n;")
    queryNeo4j(driver, "DROP CONSTRAINT n10s_unique_uri;")
    queryNeo4j(driver, "CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE;")
    queryNeo4j(driver, "CALL n10s.graphconfig.init({handleVocabUris: \"SHORTEN\", keepLangTag: false, handleMultival: \"ARRAY\"});")
    queryNeo4j(driver, importQuery)
    queryNeo4j(driver, """
    MATCH (n)
    WHERE n.uri STARTS WITH 'http://w3id.org/annett-o/'
    SET n.uri = SPLIT(n.uri, '/')[SIZE(SPLIT(n.uri, '/')) - 1]
    """)

import_ontology_to_neo4j()
        